In [38]:
# Cell 0
# If at all possible please test locally or on the private tbears server. The Testnet
# is becoming cluttered with many deployments of Balanced contracts.
# Note that running on the private tbears server will require the number of top P-Reps 
# be set to 4 in the staking contract or it will fail to deploy.

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"gangnam": {"iconservice": "https://gicon.net.solidwallet.io", "nid": 3},
"hannam": {"iconservice": "https://hannam.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [39]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
TEST_ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"
MAIN_ORACLE = "cxe647e0af68a4661566f5e9861ad4ac854de808a2"
BALANCED_TEST = "hx3f01840a599da07b0f620eeae7aa9c574169a4be"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [40]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [41]:
# Cell 3

wallet = KeyWallet.load("../../keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("../../keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("../../keystores/balanced_test.json", key_data)
with open("../../keystores/staking_test.pwd", "r") as f:
    key_data = f.read()
staking_wallet = KeyWallet.load("../../keystores/staking_test.json", key_data)


In [42]:
print(wallet.get_address())
print(icon_service.get_balance(wallet.get_address()) / 10**18)

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb
779546486.6654812


In [43]:
print(btest_wallet.get_address())
print(icon_service.get_balance(btest_wallet.get_address()) / 10**18)

hx3f01840a599da07b0f620eeae7aa9c574169a4be
1945224.5162162


In [44]:
user1 = KeyWallet.load("../../keystores/user1.json","HelloWorld@1234")
# btest_wallet = KeyWallet.load("./balanced_test.json","HelloWorld@1234")

print(icon_service.get_balance(user1.get_address())/10**18)
print(user1.get_address())

# test2 = hx7a1824129a8fe803e45a3aae1c0e060399546187
private = "0a354424b20a7e3c55c43808d607bddfac85d033e63d7d093cb9f0a26c4ee022"
user2 = KeyWallet.load(bytes.fromhex(private))
print(icon_service.get_balance(user2.get_address())/10**18)
print(user2.get_address())

181999.8088096
hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db
206479.709028
hx7a1824129a8fe803e45a3aae1c0e060399546187


In [47]:
dict_list = []
for i in range(0, 50):
    data = {}
    wallet2 = KeyWallet.create()
    address = wallet2.get_address()
    private_key = wallet2.get_private_key()
    data[address] = private_key
    dict_list.append(data)
with open('wallet2.json', 'a') as f:
    json.dump(dict_list, f, ensure_ascii=False, indent=4)

In [48]:
with open('wallet2.json') as f:
    data = json.load(f)

In [49]:
for dict in data:
    for items in dict.items():
        print(items[0])
        set_div_per = TransactionBuilder() \
            .from_(wallet.get_address()) \
            .to(items[0]) \
            .nid(NID) \
            .nonce(100) \
            .value(5000 * 10 ** 18) \
            .build()
        estimate_step = icon_service.estimate_step(set_div_per)
        step_limit = estimate_step + 10000000
        signed_transaction = SignedTransaction(set_div_per, wallet, step_limit)

        tx_hash = icon_service.send_transaction(signed_transaction)
        ab = get_tx_result(tx_hash)

hx842ea463c96e2926693986f0b71a1c966edece03
hx55c778bccee681f1ad32b9759d5f77d22f796350
hxf2c66c3979c95fabc4b85b072d42bc489171fd4d


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
hxd3c4b7c04eafd6ca388810f436843a96a2a35798


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
hx7389bc58eea5a3a94283c815cb2a8d9824e43bc9


...

Retrying in 1 seconds...
hx4a090f08b983c250c18fb191f6e5e1b597a32043
hx11692ca1d89f0b34fec62d97f7a7a3c0629ef24c


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
hxb6b193a2cf741bd75a4c3cfc6322e58b5b3f4522


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
hx970f9feecadfadca5c9fda3bf80c38d90cbcef1e


In [9]:
contracts = {'loans': {'zip': '../../core_contracts/loans.zip', 'SCORE': 'cxe38b01784e58f72d75aa8450bdee22d62fbdb054'}, 'staking': {'zip': '../../core_contracts/staking.zip', 'SCORE': 'cxd37f220d61294c14e7d38d5e8f723c6c5a81d994'}, 'dividends': {'zip': '../../core_contracts/dividends.zip', 'SCORE': 'cx3144c3c8ea295653a8f92e83b9d6b0f2c3457e2c'}, 'reserve': {'zip': '../../core_contracts/reserve.zip', 'SCORE': 'cxf8058e6dca2dae0a7a2aa470cde7d5e91bb0c562'}, 'daofund': {'zip': '../../core_contracts/daofund.zip', 'SCORE': 'cx50f12e56855d5ad266304fec71603a0eda638735'}, 'rewards': {'zip': '../../core_contracts/rewards.zip', 'SCORE': 'cx2ef5f06ac02d861e735222cc0718c83d2419405f'}, 'dex': {'zip': '../../core_contracts/dex.zip', 'SCORE': 'cxd867d7843d6a489b23392719435c70ec3442161a'}, 'governance': {'zip': '../../core_contracts/governance.zip', 'SCORE': 'cx53a614e3d3ce7d8ee01920cba52d9c8f53fd0f6d'}, 'oracle': {'zip': '../../core_contracts/oracle.zip', 'SCORE': 'cxed97bdb35a7ca1b3993e400e4dba9e11610338f7'}, 'sicx': {'zip': '../../token_contracts/sicx.zip', 'SCORE': 'cx313a7b07cd2cb470f6baaf4aaccecf925456a3ca'}, 'bnUSD': {'zip': '../../token_contracts/bnUSD.zip', 'SCORE': 'cxb0e4035799bc4e61230713a6fedde99924524736'}, 'bnXLM': {'zip': '../../token_contracts/bnXLM.zip', 'SCORE': 'cx9e4e0161c712e36c86d7a85f8c39ce45a5616944'}, 'bnDOGE': {'zip': '../../token_contracts/bnDOGE.zip', 'SCORE': 'cxed2a1ca56302958c5aa084191a367d374337e32c'}, 'baln': {'zip': '../../token_contracts/baln.zip', 'SCORE': 'cx6c096b9e451d1f7e5f6e4016a992d2a892a9053a'}, 'bwt': {'zip': '../../token_contracts/bwt.zip', 'SCORE': 'cx43e5e722f6607396696f3a57e9bd98ea4f21b5ac'}}

In [50]:
# Cell 5a
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxa0f715fb2c4bc8f4c6399c2cc26167a27be0aa61'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxbabed822d59b605dbeb6322735c529b292baac3b'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx1379084f45776301abda3849c6e374f460ee0155'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx71dda2221bf88faddc8f84b72ffd6db296e5609e'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxfd09787f23d23b945fa0c7eb55b5aa69425da1c8'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx27aa3bf62145822e60d85fa5d18dabdcff5b9ada'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx01eee12b6614e5328e0a84261652cb7f055e0176'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxd7b3e71dcff3d75392216e208f28ef68e8a54ec0'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cxed97bdb35a7ca1b3993e400e4dba9e11610338f7'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx799f724e02560a762b5f2bd3b6d2d8d59d7aecc1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx4d0768508a7ff550de4405f27aebfb8831565c19'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx663f9d59163846d9f6c6f7b586858c59aa8878a9'}}

In [51]:
# Cell 6
# Define deploy and send_tx functions

def send_icx(to, amount, wallet):
    transaction = TransactionBuilder()\
        .from_(wallet.get_address())\
        .to(to)\
        .value(amount)\
        .step_limit(1000000) \
        .nid(NID) \
        .nonce(2) \
        .version(3) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    return icon_service.send_transaction(signed_transaction)


def compress():
    """
    Compress all SCORE folders in the core_contracts and token_contracts folders.
    Make sure the oracle address is correct.
    """
    deploy = list(contracts.keys())[:]
    for directory in {"../../core_contracts", "../../token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'
    if network == "yeouido":
        contracts['oracle']['SCORE'] = TEST_ORACLE
    elif network == "mainnet":
        contracts['oracle']['SCORE'] = MAIN_ORACLE
                    

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 4000100000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res

def send_tx(dest, value, method, params, wallet, _print = True):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Calling {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    return res

def deploy_all(wallet, staking_wallet):
    """
    Compress and Deploy all SCOREs.
    """
    compress()

    deploy = list(contracts.keys())[:]
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')
    if network == "mainnet":
        deploy.remove('bnXLM')
        deploy.remove('bnDOGE')

    results = {}
    res = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    results[f'{contracts["governance"]}|deploy|{{}}'] = res
    governance = res.get('scoreAddress', '')
    contracts['governance']['SCORE'] = governance
    params = {'_governance': governance}
    for score in deploy:
        res = deploy_SCORE(contracts[score], params, wallet, 0)
        results[f'{contracts[score]}|deploy|{params}'] = res
        contracts[score]['SCORE'] = res.get('scoreAddress', '')

    res = deploy_SCORE(contracts['staking'], {}, staking_wallet, 0)
    results[f'{contracts["staking"]}|deploy|{{}}'] = res
    contracts['staking']['SCORE'] = res.get('scoreAddress', '')

    params = {'_admin': contracts['staking']['SCORE']}
    res = deploy_SCORE(contracts['sicx'], params, staking_wallet, 0)
    results[f'{contracts["sicx"]}|deploy|{params}'] = res
    contracts['sicx']['SCORE'] = res.get('scoreAddress', '')
    
    return results

    
def config_balanced(wallet, staking_wallet):
    """
    Configure all SCOREs before launch.
    """
    config = list(contracts.keys())[:]
    config.remove('governance')
    config.remove('bnDOGE')
    config.remove('bnXLM')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}
    txns = [{'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'configureBalanced', 'params': {}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results

def launch_balanced(wallet, staking_wallet):
    """
    Launch Balanced, turn on staking management, and set delegation for sICX on the Loans contract.
    """
    if network == "custom":
        preps = {
            "hx9eec61296a7010c867ce24c20e69588e2832bc52",  # ICX Station
            "hx000e0415037ae871184b2c7154e5924ef2bc075e"}  # iBriz-ICONOsphere
    elif network == "yeouido":
        preps = {
            "hx23823847f593ecb65c9e1ea81a789b02766280e8",  # ICX Station
            "hxe0cde6567eb6529fe31b0dc2f2697af84847f321",  # iBriz-ICONOsphere
            "hx83c0fc2bcac7ecb3928539e0256e29fc371b5078",  # Mousebelt
            "hx48b4636e84d8c491c88c18b65dceb7598c4600cc",  # Parrot 9
            "hxb4e90a285a79687ec148c29faabe6f71afa8a066"}  # ICONDAO
    elif network == "mainnet":
        preps = {
            "",  # ICX Station
            "",  # iBriz-ICONOsphere
            "",  # Mousebelt
            "",  # Parrot 9
            ""}  # ICONDAO
    else:
        return
    
    txns = [{'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'toggleStakingOn', 'params': {}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'delegate', 'params': {'_delegations': [{'_address': prep, '_votes_in_per': str(100 * ICX // len(preps))} for prep in preps]}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results
   

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}, _print = True):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Reading {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    result = icon_service.call(call)
    print(result)
    return result

In [52]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

if network == 'custom':
    confirm = 'Yes'
else:
    confirm = input(f'Deploying Balanced to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    results = {}
    deploy_all(btest_wallet, staking_wallet)
    print('------------------------------------------------------------------------------------------------------------------')
    print(contracts)
    print('----------Contracts for Testing UI--------------------------------------------------------------------------------')
    print(get_scores_json(contracts))

../../core_contracts/governance.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/loans.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/dividends.zip


...

Retrying in 1 seconds...
Status: 1

../../token_contracts/bwt.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/staking.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../token_contracts/sicx.zip
Status: 1

------------------------------------------------------------------------------------------------------------------
{'loans': {'zip': '../../core_contracts/loans.zip', 'SCORE': 'cx30b8d687d71da2ed61df8524dac13913b371732c'}, 'staking': {'zip': '../../core_contracts/staking.zip', 'SCORE': 'cx50a223e1e3ca176424e55ff2edfb9f1a0148ff1b'}, 'dividends': {'zip': '../../core_contracts/dividends.zip', 'SCORE': 'cx95894b45f14d3f7952f010027e17d63724578eb3'}, 'reserve': {'zip': '../../core_contracts/reserve.zip', 'SCORE': 'cxc3d786dc0e36d07bd7a0041a94841801fcbe01f7'}, 'daofund': {'zip': '../../core_contracts/daofund.zip', 'SCORE': 'cx13707b8e538651274dfecf97481658873dafb5f5'}, 'rewards': {'zip': '../../core_contracts/rewards.zip', 'SCORE': 'cx4843bdc51a5e922a4d5cd2f927aa4421fec96d99'}, 'dex': {'zip': '../../core_contracts/dex.zip', 'SCORE': 'cxead57124302bdf1241cb798d43ff41214f4fe822'}, 'governance': {'zip': '../../core_con

In [53]:
# Cell 7a
# Configure Balanced

config_results = config_balanced(btest_wallet, staking_wallet)
print(config_results)

------------------------------------------------------------------------------------------------------------------
Calling setAddresses, with parameters {'_addresses': {'loans': 'cx30b8d687d71da2ed61df8524dac13913b371732c', 'staking': 'cx50a223e1e3ca176424e55ff2edfb9f1a0148ff1b', 'dividends': 'cx95894b45f14d3f7952f010027e17d63724578eb3', 'reserve': 'cxc3d786dc0e36d07bd7a0041a94841801fcbe01f7', 'daofund': 'cx13707b8e538651274dfecf97481658873dafb5f5', 'rewards': 'cx4843bdc51a5e922a4d5cd2f927aa4421fec96d99', 'dex': 'cxead57124302bdf1241cb798d43ff41214f4fe822', 'oracle': 'cxed97bdb35a7ca1b3993e400e4dba9e11610338f7', 'sicx': 'cx45800addcd1a03da7ce7f2029ccb61005556579d', 'bnUSD': 'cx1a6e81deeb45f5c5f465e352c8966c2dd1b664b5', 'baln': 'cxa65ff475728cbf98989ff3d36e6ac49d9fe2a734', 'bwt': 'cx293b2d65e45cd6ab0d827c96fd88ca169893a206'}} on the governance contract.
------------------------------------------------------------------------------------------------------------------
********************

In [54]:
# Cell 7b
# Launch Balanced
# We may want to make this a payable method and have the governance SCORE borrow bnUSD,
# start and name the sICXbnUSD market, and add it as a rewards DataSource.

launch_results = launch_balanced(btest_wallet, staking_wallet)
print(launch_results)

------------------------------------------------------------------------------------------------------------------
Calling launchBalanced, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx30b8d687d71da2ed61df8524dac13913b371732c', 'indexed': ['ContractActive(str,str)', 'Loans'], 'data': ['Active']} 

------------------------------------------------------------------------------------------------------------------
Calling toggleStakingOn, with parameters {} on the staking contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling delegate, with parameters {'_delegations': [{'_address': 'hx9eec61296a7010c867ce24c20e69588e2832bc52', '_votes_in_per': '50000000000000000000'}, {'_address': 'hx000e0415037ae871184b2c7154e5924ef2bc075e', '_votes_in_per': '50000000000000000000'}]} on the governance contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'governance|launchBalanced|{}': {'txHash': '0x80641fffadac1822cdf40eb845c86b74eef845e3f6f7b5a795c94abee5cabfa1', 'blockHeight': 559866, 'blockHash': '0x49dc99d64ca607dca28c000ae86e36f7e4afcf90c56ad7cb3bb320b7d87d53c1', 'txIndex': 1, 'to'

In [34]:
# Cell 8
# Deploy or Update a single SCORE

contract_name = 'rewards'
update = 1
params = {}
if update == 0:
    params = {'_governance': contracts['governance']['SCORE']}

compress()
contract = contracts[contract_name]
confirm = input(f'{"Updating" if update else "Deploying"} {contract_name} with params: {params} to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    deploy_SCORE(contract, params, btest_wallet, update)

Updating rewards with params: {} to custom. Proceed (Yes/No)? Yes
../../core_contracts/rewards.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1



In [57]:
for dict in data:
    for items in dict.items():
        print(items[0])
        send_tx('loans', 300*ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 10* ICX}, KeyWallet.load(bytes.fromhex(items[1])))


hx842ea463c96e2926693986f0b71a1c966edece03
------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 10000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx30b8d687d71da2ed61df8524dac13913b371732c', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx30b8d687d71da2ed61df8524dac13913b371732c', 'cx50a223e1e3ca176424e55ff2edfb9f1a0148ff1b', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx45800addcd1a03da7ce7f2029ccb61005556579d', 'indexed': ['Mint(Address,int,bytes)', 'cx30b8d687d71da2ed61df8524dac13913b371732c'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx45800addcd1a03da7ce7f2029ccb61005556579d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cx30b8d687d71da2ed61df8524dac13913b371732c', '0x1043561a8829300000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx50a223e1e3ca176424e55ff2edfb9f1a0148ff1b', 'indexed': ['TokenTransfer(Address,int,str)', 'cx30b8d687d71da2ed61df8524dac13913b371732c', '0x1043561

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx30b8d687d71da2ed61df8524dac13913b371732c', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx30b8d687d71da2ed61df8524dac13913b371732c', 'cx50a223e1e3ca176424e55ff2edfb9f1a0148ff1b', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx45800addcd1a03da7ce7f2029ccb61005556579d', 'indexed': ['Mint(Address,int,bytes)', 'cx30b8d687d71da2ed61df8524dac13913b371732c'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx45800addcd1a03da7ce7f2029ccb61005556579d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cx30b8d687d71da2ed61df8524dac13913b371732c', '0x1043561a8829300000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx50a223e1e3ca176424e55ff2edfb9f1a0148ff1b', 'indexed': ['TokenTransfer(Address,int,str)', 'cx30b8d687d71da2ed61df8524dac13913b371732c', '0x1043561

...

Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx30b8d687d71da2ed61df8524dac13913b371732c', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx30b8d687d71da2ed61df8524dac13913b371732c', 'cx50a223e1e3ca176424e55ff2edfb9f1a0148ff1b', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx45800addcd1a03da7ce7f2029ccb61005556579d', 'indexed': ['Mint(Address,int,bytes)', 'cx30b8d687d71da2ed61df8524dac13913b371732c'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx45800addcd1a03da7ce7f2029ccb61005556579d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cx30b8d687d71da2ed61df8524dac13913b371732c', '0x1043561a8829300000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx50a223e1e3ca176424e55ff2edfb9f1a0148ff1b', 'indexed': ['TokenTransfer(Address,int,str)', 'cx30b8d687d71da2ed61df8524dac13913b371732c', '0x1043561

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx30b8d687d71da2ed61df8524dac13913b371732c', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx30b8d687d71da2ed61df8524dac13913b371732c', 'cx50a223e1e3ca176424e55ff2edfb9f1a0148ff1b', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx45800addcd1a03da7ce7f2029ccb61005556579d', 'indexed': ['Mint(Address,int,bytes)', 'cx30b8d687d71da2ed61df8524dac13913b371732c'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx45800addcd1a03da7ce7f2029ccb61005556579d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cx30b8d687d71da2ed61df8524dac13913b371732c', '0x1043561a8829300000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx50a223e1e3ca176424e55ff2edfb9f1a0148ff1b', 'indexed': ['TokenTransfer(Address,int,str)', 'cx30b8d687d71da2ed61df8524dac13913b371732c', '0x1043561

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx30b8d687d71da2ed61df8524dac13913b371732c', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx30b8d687d71da2ed61df8524dac13913b371732c', 'cx50a223e1e3ca176424e55ff2edfb9f1a0148ff1b', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx45800addcd1a03da7ce7f2029ccb61005556579d', 'indexed': ['Mint(Address,int,bytes)', 'cx30b8d687d71da2ed61df8524dac13913b371732c'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx45800addcd1a03da7ce7f2029ccb61005556579d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cx30b8d687d71da2ed61df8524dac13913b371732c', '0x1043561a8829300000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx50a223e1e3ca176424e55ff2edfb9f1a0148ff1b', 'indexed': ['TokenTransfer(Address,int,str)', 'cx30b8d687d71da2ed61df8524dac13913b371732c', '0x1043561

In [64]:
send_tx('loans', 500*ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 10* ICX}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 10000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx30b8d687d71da2ed61df8524dac13913b371732c', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx30b8d687d71da2ed61df8524dac13913b371732c', 'cx50a223e1e3ca176424e55ff2edfb9f1a0148ff1b', '0x1b1ae4d6e2ef500000'], 'data': []} 

{'scoreAddress': 'cx45800addcd1a03da7ce7f2029ccb61005556579d', 'indexed': ['Mint(Address,int,bytes)', 'cx30b8d687d71da2ed61df8524dac13913b371732c'], 'data': ['0x1b1ae4d6e2ef500000', '0x4e6f6e65']} 

{'scoreAddress': 'cx45800addcd1a03da7ce7f2029ccb61005556579d', 'indexed': ['Transfer(Address,Addres

{'txHash': '0xe35c699d4dfb706114b78ed1edc5eb7e7267d7050444650ee4d4c3cfd3088d3a',
 'blockHeight': 560454,
 'blockHash': '0x884bfcc7ee0252cd83393a56e5409cb3edec2fd7bc3541aa31fd551e5cf59584',
 'txIndex': 1,
 'to': 'cx30b8d687d71da2ed61df8524dac13913b371732c',
 'stepUsed': 1320300,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1320300,
 'eventLogs': [{'scoreAddress': 'cx30b8d687d71da2ed61df8524dac13913b371732c',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx30b8d687d71da2ed61df8524dac13913b371732c',
    'cx50a223e1e3ca176424e55ff2edfb9f1a0148ff1b',
    '0x1b1ae4d6e2ef500000'],
   'data': []},
  {'scoreAddress': 'cx45800addcd1a03da7ce7f2029ccb61005556579d',
   'indexed': ['Mint(Address,int,bytes)',
    'cx30b8d687d71da2ed61df8524dac13913b371732c'],
   'data': ['0x1b1ae4d6e2ef500000', '0x4e6f6e65']},
  {'scoreAddress': 'cx45800addcd1a03da7ce7f2029ccb61005556579d',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx0000000000000000000000000000000000000000',
    'cx30

In [65]:
for dict in data:
    for items in dict.items():
        print(items[0])
        call_tx('loans', 'getAccountPositions', {'_owner': items[0]})


hx842ea463c96e2926693986f0b71a1c966edece03
------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx842ea463c96e2926693986f0b71a1c966edece03'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5c12ba18cf82c', 'address': 'hx842ea463c96e2926693986f0b71a1c966edece03', 'snap_id': '0x215', 'snaps_length': '0x1', 'last_snap': '0x215', 'first day': '0x215', 'assets': {'sICX': '0x1043561a8829300000', 'bnUSD': '0x8c2a687ce7720000'}, 'total_debt': '0x53d01766d40ecfc7', 'collateral': '0x1043561a8829300000', 'ratio': '0x2b15e32a528eadc96', 'standing': 'Not Mining'}
hx55c778bccee681f1ad32b9759d5f77d22f796350
------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters 

KeyboardInterrupt: 

In [66]:
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x33', 'created': '0x5c12bdee3d180', 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'snap_id': '0x26c', 'snaps_length': '0x2', 'last_snap': '0x26c', 'first day': '0x268', 'assets': {'sICX': '0x2b5e3af16b18800000', 'bnUSD': '0x11854d0f9cee40000'}, 'total_debt': '0xa7a02ecda81d9f8e', 'collateral': '0x2b5e3af16b18800000', 'ratio': '0x397284386e139261e', 'standing': 'Not Mining'}


{'pos_id': '0x33',
 'created': '0x5c12bdee3d180',
 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
 'snap_id': '0x26c',
 'snaps_length': '0x2',
 'last_snap': '0x26c',
 'first day': '0x268',
 'assets': {'sICX': '0x2b5e3af16b18800000', 'bnUSD': '0x11854d0f9cee40000'},
 'total_debt': '0xa7a02ecda81d9f8e',
 'collateral': '0x2b5e3af16b18800000',
 'ratio': '0x397284386e139261e',
 'standing': 'Not Mining'}

In [58]:
wallets = []
for dict in data:
    for items in dict.items():
        wallets.append(items[0])
params = {'_holders': wallets}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['rewards']['SCORE'])\
                    .method("getBalnHoldings")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

{'hx842ea463c96e2926693986f0b71a1c966edece03': '0x0',
 'hx55c778bccee681f1ad32b9759d5f77d22f796350': '0x0',
 'hxf2c66c3979c95fabc4b85b072d42bc489171fd4d': '0x0',
 'hxd3c4b7c04eafd6ca388810f436843a96a2a35798': '0x0',
 'hx7389bc58eea5a3a94283c815cb2a8d9824e43bc9': '0x0',
 'hx85d649746c44465f135673a6b8228257664a383e': '0x0',
 'hx1481062a7efbbaf9bf84d108d05f4096de18c4be': '0x0',
 'hx64deb08f01ced67aa1c390513e0d46b70321c77a': '0x0',
 'hx4fe84a69bcc9e8446d93ce9e3ba5f6102b1dcab0': '0x0',
 'hxcc949262b52d221679da73172c85afcdc6270169': '0x0',
 'hxe21ac0fca9437d6f2b7eb64602e208933650aae2': '0x0',
 'hx339de2122199ffca03dc9ece99f9b24f96ee17f7': '0x0',
 'hxb8f5421f9f4816ac23b92f4ff0728d05cd08562d': '0x0',
 'hx4f5b44ba33ef3633ff4bc9c4eb62b15fa1934a29': '0x0',
 'hxf4d631f12f697b7e6e114e794f05841a0819ccd8': '0x0',
 'hxd9e820303ecaeeeb9d4a24ff823da490a78cd350': '0x0',
 'hxcca5457d39b1e18eb8dc18d3e98cc0e9033702ec': '0x0',
 'hx5c9ef20d179db9ac4d459513a3129038c792532b': '0x0',
 'hx26f7348d5ee7c0d02c69a72e

In [59]:
# Check BALN that has been distributed on the rewards SCORE, and to platform contract addresses.

borrowerCount = int(call_tx('loans', 'borrowerCount', {}, False), 0)

addresses = []
for i in range(1, borrowerCount + 1):
    position = call_tx('loans', 'getPositionByIndex', {'_index': i, '_day': -1}, False)
    addresses.append(position['address'])

holders = call_tx('rewards', 'getBalnHoldings', {'_holders': addresses}, False)

total_balances = 0
baln_balances = {}
for contract in ['rewards', 'reserve', 'bwt', 'dex', 'daofund']:
    result = int(call_tx('baln', 'balanceOf', {'_owner': contracts[contract]['SCORE']}, False), 0)
    baln_balances[contract] = result / 10**18
    total_balances += result

i = 0
holdings = {i: [key, int(holders[key], 0), int(holders[key], 0) / 10**18] for i, key in enumerate(holders.keys())}
total = 0
for key in holdings:
    total += holdings[key][1]
    print(f'{holdings[key]}')

print(f'Total unclaimed: {total / 10**18}')
print(baln_balances)
print(f'Total BALN: {total_balances / 10**18}')

------------------------------------------------------------------------------------------------------------------
Reading borrowerCount, with parameters {} on the loans contract.
------------------------------------------------------------------------------------------------------------------
0x32
------------------------------------------------------------------------------------------------------------------
Reading getPositionByIndex, with parameters {'_index': 1, '_day': -1} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5c12ba18cf82c', 'address': 'hx842ea463c96e2926693986f0b71a1c966edece03', 'snap_id': '0x215', 'snaps_length': '0x1', 'last_snap': '0x215', 'first day': '0x215', 'assets': {'sICX': '0x1043561a8829300000', 'bnUSD': '0x8c2a687ce7720000'}, 'total_debt': '0x53d01766d40ecfc7', 'collateral': '0x1043561a8829300000', 'ratio': '0x2b15e32a528eadc96', 'sta

{'pos_id': '0xc', 'created': '0x5c12ba319f930', 'address': 'hx339de2122199ffca03dc9ece99f9b24f96ee17f7', 'snap_id': '0x217', 'snaps_length': '0x1', 'last_snap': '0x217', 'first day': '0x217', 'assets': {'sICX': '0x1043561a8829300000', 'bnUSD': '0x8c2a687ce7720000'}, 'total_debt': '0x53d01766d40ecfc7', 'collateral': '0x1043561a8829300000', 'ratio': '0x2b15e32a528eadc96', 'standing': 'Not Mining'}
------------------------------------------------------------------------------------------------------------------
Reading getPositionByIndex, with parameters {'_index': 13, '_day': -1} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0xd', 'created': '0x5c12ba33880d8', 'address': 'hxb8f5421f9f4816ac23b92f4ff0728d05cd08562d', 'snap_id': '0x217', 'snaps_length': '0x1', 'last_snap': '0x217', 'first day': '0x217', 'assets': {'sICX': '0x1043561a8829300000', 'bnUSD': '0x8c2a687ce7720000'}, 'total_deb

{'pos_id': '0x18', 'created': '0x5c12ba4a6ea00', 'address': 'hx3f6eba4615249458ef55ea4fcbda140381f0e43f', 'snap_id': '0x219', 'snaps_length': '0x1', 'last_snap': '0x219', 'first day': '0x219', 'assets': {'sICX': '0x1043561a8829300000', 'bnUSD': '0x8c2a687ce7720000'}, 'total_debt': '0x53d01766d40ecfc7', 'collateral': '0x1043561a8829300000', 'ratio': '0x2b15e32a528eadc96', 'standing': 'Not Mining'}
------------------------------------------------------------------------------------------------------------------
Reading getPositionByIndex, with parameters {'_index': 25, '_day': -1} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x19', 'created': '0x5c12ba4c5719a', 'address': 'hx2d6b58869aac1faa6b799f002b3b77842ade3252', 'snap_id': '0x21a', 'snaps_length': '0x1', 'last_snap': '0x21a', 'first day': '0x21a', 'assets': {'sICX': '0x1043561a8829300000', 'bnUSD': '0x8c2a687ce7720000'}, 'total_d

{'pos_id': '0x24', 'created': '0x5c12ba6ae0222', 'address': 'hx5258c630615204446827dd75a0202bb6c3c4af2b', 'snap_id': '0x21c', 'snaps_length': '0x1', 'last_snap': '0x21c', 'first day': '0x21c', 'assets': {'sICX': '0x1043561a8829300000', 'bnUSD': '0x8c2a687ce7720000'}, 'total_debt': '0x53d01766d40ecfc7', 'collateral': '0x1043561a8829300000', 'ratio': '0x2b15e32a528eadc96', 'standing': 'Not Mining'}
------------------------------------------------------------------------------------------------------------------
Reading getPositionByIndex, with parameters {'_index': 37, '_day': -1} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x25', 'created': '0x5c12ba6cc89da', 'address': 'hx979fc58becb5bd542db447032403829e935a68c1', 'snap_id': '0x21c', 'snaps_length': '0x1', 'last_snap': '0x21c', 'first day': '0x21c', 'assets': {'sICX': '0x1043561a8829300000', 'bnUSD': '0x8c2a687ce7720000'}, 'total_d

{'pos_id': '0x30', 'created': '0x5c12ba85979e3', 'address': 'hx11692ca1d89f0b34fec62d97f7a7a3c0629ef24c', 'snap_id': '0x21e', 'snaps_length': '0x1', 'last_snap': '0x21e', 'first day': '0x21e', 'assets': {'sICX': '0x1043561a8829300000', 'bnUSD': '0x8c2a687ce7720000'}, 'total_debt': '0x53d01766d40ecfc7', 'collateral': '0x1043561a8829300000', 'ratio': '0x2b15e32a528eadc96', 'standing': 'Not Mining'}
------------------------------------------------------------------------------------------------------------------
Reading getPositionByIndex, with parameters {'_index': 49, '_day': -1} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x31', 'created': '0x5c12ba878035d', 'address': 'hxb6b193a2cf741bd75a4c3cfc6322e58b5b3f4522', 'snap_id': '0x21f', 'snaps_length': '0x1', 'last_snap': '0x21f', 'first day': '0x21f', 'assets': {'sICX': '0x1043561a8829300000', 'bnUSD': '0x8c2a687ce7720000'}, 'total_d

0x0
------------------------------------------------------------------------------------------------------------------
Reading balanceOf, with parameters {'_owner': 'cx13707b8e538651274dfecf97481658873dafb5f5'} on the baln contract.
------------------------------------------------------------------------------------------------------------------
0x19f0bd01a72f079000000
['hx842ea463c96e2926693986f0b71a1c966edece03', 0, 0.0]
['hx55c778bccee681f1ad32b9759d5f77d22f796350', 0, 0.0]
['hxf2c66c3979c95fabc4b85b072d42bc489171fd4d', 0, 0.0]
['hxd3c4b7c04eafd6ca388810f436843a96a2a35798', 0, 0.0]
['hx7389bc58eea5a3a94283c815cb2a8d9824e43bc9', 0, 0.0]
['hx85d649746c44465f135673a6b8228257664a383e', 0, 0.0]
['hx1481062a7efbbaf9bf84d108d05f4096de18c4be', 0, 0.0]
['hx64deb08f01ced67aa1c390513e0d46b70321c77a', 0, 0.0]
['hx4fe84a69bcc9e8446d93ce9e3ba5f6102b1dcab0', 0, 0.0]
['hxcc949262b52d221679da73172c85afcdc6270169', 0, 0.0]
['hxe21ac0fca9437d6f2b7eb64602e208933650aae2', 0, 0.0]
['hx339de2122199ffca03d